# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

A. d. Graaff  ->  A. D. Graaff  |  ['A. D. Graaff']
H.-W. Rix  ->  H.-W. Rix  |  ['H.-W. Rix']
F. Walter  ->  F. Walter  |  ['F. Walter']
J. Shi  ->  J. Shi  |  ['J. Shi']
L. Xie  ->  Z.-L. Xie  |  ['L. Xie']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
H. Jiang  ->  H. Jiang  |  ['H. Jiang']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']


M. Bergemann  ->  M. Bergemann  |  ['M. Bergemann']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
H. Jiang  ->  H. Jiang  |  ['H. Jiang']
D. M. Fuksman  ->  D. M. Fuksman  |  ['D. M. Fuksman']
J. Baier  ->  J. Baier  |  ['J. Baier']
K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
G. Chauvin  ->  G. Chauvin  |  ['G. Chauvin']


Arxiv has 110 new papers today
          14 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/14 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2511.21820


extracting tarball to tmp_2511.21820...

 done.


Retrieving document from  https://arxiv.org/e-print/2511.22071


...

/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found 10 candidates with documentclass definition.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Assuming tmp_2511.22732/AandA.tex as main document.
  exec(code_obj, self.user_global_ns, self.user_ns)


 done.
Retrieving document from  https://arxiv.org/e-print/2511.23163
extracting tarball to tmp_2511.23163...

/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2511.23147/SOmanuscript.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'new-tab3.tex' from 'tmp_2511.23147/new-tab3.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


 done.


Issues with the citations
repeated bibliography entry: pierrehumbert2010


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.21820-b31b1b.svg)](https://arxiv.org/abs/2511.21820) | **Little Red Dots host Black Hole Stars: A unified family of gas-reddened AGN revealed by JWST/NIRSpec spectroscopy**  |
|| <mark>A. d. Graaff</mark>, et al. -- incl., <mark>H.-W. Rix</mark>, <mark>F. Walter</mark> |
|*Appeared on*| *2025-12-01*|
|*Comments*| *22 pages, 18 figures, submitted to MNRAS. LRD sample and measurements are publicly available at this https URL*|
|**Abstract**|            We use the DAWN JWST Archive to construct and characterise a sample of 116 little red dots (LRDs) across 2.3<z<9.3, selecting all sources with v-shaped UV-optical continua from NIRSpec/PRISM spectra and compact morphologies in NIRCam/F444W imaging. We show that LRD continuum spectra are ubiquitously well described by modified blackbodies across ~$0.4-1.0\mu$m, with typical T~5000K or $\lambda_{peak}$~$0.65\mu$m across 2 dex in luminosity, and a tail toward T~2000K. LRDs therefore trace a locus in the Hertzsprung-Russell diagram that is directly analogous to stars on the Hayashi track, strongly supporting the picture that LRDs are AGN embedded in thermalised dense gas envelopes in approximate hydrostatic equilibrium. Hotter LRDs with $\lambda_{peak}<0.65\mu$m typically have strong Balmer breaks, redder UV slopes and high optical luminosities; other LRDs show weak or no Balmer breaks, and wide variety in $\beta_{UV}$ and $L_{5100}$. Crucially, we demonstrate that the UV-optical continuum shapes and luminosities are strongly linked to the $H\alpha,\ H\beta$, [OIII] and OI line properties. There is a tight linear relation between the H$\alpha$ and optical continuum luminosities, as well as H$\alpha$ and OI$_{8446}$, indicating that Balmer, OI and optical emission must primarily be powered by the same source. The Balmer decrement increases strongly toward higher $L_{H\alpha}$, $L_{5100}$ and Balmer break strength, providing key evidence for luminosity-dependent effects of collisional (de-)excitation and resonant scattering in the gaseous envelopes. In contrast, we show that [OIII] emission likely originates from star-forming host galaxies, and that its strong correlation with Balmer break strength arises naturally from variation in the AGN-to-host ratio. Our work presents an empirical description of the nature and structure of LRDs, defining a new benchmark for ongoing LRD model developments.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.22399-b31b1b.svg)](https://arxiv.org/abs/2511.22399) | **Euclid preparation: LXXXI. The impact of nonparametric star formation histories on spatially resolved galaxy property estimation using synthetic Euclid images**  |
|| E. Collaboration, et al. -- incl., <mark>K. Jahnke</mark> |
|*Appeared on*| *2025-12-01*|
|*Comments*| *23 pages, 12 figures, accepted*|
|**Abstract**|            We analyzed the spatially resolved and global star formation histories (SFHs) for a sample of 25 TNG50-SKIRT Atlas galaxies to assess the feasibility of reconstructing accurate SFHs from Euclid-like data. This study provides a proof of concept for extracting the spatially resolved SFHs of local galaxies with Euclid, highlighting the strengths and limitations of SFH modeling in the context of next-generation galaxy surveys. We used the spectral energy distribution (SED) fitting code Prospector to model both spatially resolved and global SFHs using parametric and nonparametric configurations. The input consisted of mock ultraviolet--near-infrared photometry derived from the TNG50 cosmological simulation and processed with the radiative transfer code SKIRT. We show that nonparametric SFHs provide a more effective approach to mitigating the outshining effect by recent star formation, offering improved accuracy in the determination of galaxy stellar properties. Also, we find that the nonparametric SFH model at resolved scales closely recovers the stellar mass formation times (within 0.1~dex) and the ground truth values from TNG50, with an absolute average bias of $0.03$~dex in stellar mass and $0.01$~dex in both specific star formation rate and mass-weighted age. In contrast, larger offsets are estimated for all stellar properties and formation times when using a simple $\tau$-model SFH, at both resolved and global scales, highlighting its limitations. These results emphasize the critical role of nonparametric SFHs in both global and spatially resolved analyses, as they better capture the complex evolutionary pathways of galaxies and avoid the biases inherent in simple parametric models.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.22491-b31b1b.svg)](https://arxiv.org/abs/2511.22491) | **Classical Cepheids in the Galactic thin disk. I. Abundance gradients through NLTE spectral analysis**  |
|| A. Nunnari, et al. -- incl., <mark>M. Bergemann</mark> |
|*Appeared on*| *2025-12-01*|
|*Comments*| *Manuscript submitted to A&A. Abstract shortened. Comments are welcome*|
|**Abstract**|            Classical Cepheids (CCs) have long been considered excellent tracers of the chemical evolution of the Milky Way's young disk. We present a homogeneous, NLTE spectroscopic analysis of 401 Galactic CCs, based on 1,351 high-resolution optical spectra, spanning Galactocentric distances from 4.6 to 29.3 kpc. Using PySME with MARCS atmospheres and state-of-the-art grids of NLTE departure coefficients, we derive atmospheric parameters and abundances for key species tracing multiple nucleosynthetic channels. Our sample-the largest CC NLTE dataset to date-achieves high internal precision and enables robust modeling of present-day thin-disk abundance patterns and radial gradients. We estimate abundance gradients using three analytic prescriptions (linear, logarithmic, bilinear with a break) within a Bayesian, outlier-robust framework, and we also apply Gaussian Process Regression to capture non-parametric variations. We find that NLTE atmospheric parameters differ systematically from LTE determinations. Moreover, iron and most elemental abundance profiles are better described by non-linear behavior rather than by single-slope linear models: logarithmic fits generally outperform simple linear models, while bilinear fits yield inconsistent break radii across elements. Gaussian Process models reveal a consistent outer-disk flattening of [X/H] for nearly all studied elements. The [X/Fe] ratios are largely flat with Galactocentric radius, indicating coherent chemical scaling with iron across the thin disk, with modest positive offsets for Na and Al and mild declines for Mn and Cu. Comparison with recent literature shows overall agreement but highlights NLTE-driven differences, especially in outer-disk abundances. These results provide tighter empirical constraints for chemo-dynamical models of the Milky Way and set the stage for future NLTE mapping with upcoming large spectroscopic surveys.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.23163-b31b1b.svg)](https://arxiv.org/abs/2511.23163) | **Large-amplitude Variability Driven by Giant Dust Storms on a Planetary-mass Companion**  |
|| X. Tan, et al. -- incl., <mark>X. Zhang</mark>, <mark>G. Chauvin</mark> |
|*Appeared on*| *2025-12-01*|
|*Comments*| *Science Advances 2025 Nov. 26;11(48). The simulation data in this work are publicly available at this http URL*|
|**Abstract**|            Large-amplitude variations are commonly observed in the atmospheres of directly imaged exoplanets and brown dwarfs. VHS 1256B, the most variable known planet-mass object, exhibits a near-infrared flux change of nearly 40%, with red color and silicate features revealed in recent JWST spectra, challenging current theories. Using a general circulation model, we demonstrate that VHS 1256B's atmosphere is dominated by planetary-scale dust storms persisting for tens of days, with large patchy clouds propagating with equatorial waves. This weather pattern, distinct from the banded structures seen on solar system giants, simultaneously explains the observed spectra and critical features in the rotational light curves, including the large amplitude, irregular evolution, and wavelength dependence, as well as the variability trends observed in near-infrared color-magnitude diagrams of dusty substellar atmospheres.         |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.22597-b31b1b.svg)](https://arxiv.org/abs/2511.22597) | **The NANOGrav 12.5-year Data Set: Chromatic Noise Characterization & Mitigation with Time-Domain Kernels**  |
|| J. S. Hazboun, et al. -- incl., <mark>J. Baier</mark> |
|*Appeared on*| *2025-12-01*|
|*Comments*| *30 pages, 12 Figures, 6 Tables*|
|**Abstract**|            Pulsar timing arrays (PTAs) have recently entered the detection era, quickly moving beyond the goal of simply improving sensitivity at the lowest frequencies for the sake of observing the stochastic gravitational wave background (GWB), and focusing on its accurate spectral characterization. While all PTA collaborations around the world use Fourier-domain Gaussian processes to model the GWB and intrinsic long time-correlated (red) noise, techniques to model the time-correlated radio frequency-dependent (chromatic) processes have varied from collaboration to collaboration. Here we test a new class of models for PTA data, Gaussian processes based on time-domain kernels that model the statistics of the chromatic processes starting from the covariance matrix. As we will show, these models can be effectively equivalent to Fourier-domain models in mitigating chromatic noise. This work presents a method for Bayesian model selection across the various choices of kernel as well as deterministic chromatic models for non-stationary chromatic events and the solar wind. As PTAs turn towards high frequency (>1/yr) sensitivity, the size of the basis used to model these processes will need to increase, and these time-domain models present some computational efficiencies compared to Fourier-domain models.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.22732-b31b1b.svg)](https://arxiv.org/abs/2511.22732) | **EMU and Euclid: Detection of a radio-optical galaxy clustering cross-correlation signal between the Evolutionary Map of the Universe and Euclid**  |
|| G. Piccirilli, et al. -- incl., <mark>K. Jahnke</mark> |
|*Appeared on*| *2025-12-01*|
|*Comments*| **|
|**Abstract**|            Synergies between large-scale radio-continuum and optical/near-infrared galaxy surveys are a powerful tool for cosmology. Cross-correlating these surveys can constrain the redshift distribution of radio sources, mitigate systematic effects, and place constraints on cosmological models. We perform the first measurement of the clustering cross-spectrum between radio-continuum sources in the Evolutionary Map of the Universe (EMU) survey and galaxies from the ESA Euclid satellite mission's Q1 release. Our goal is to detect and characterise the cross-correlation signal, test its robustness against systematic effects, and compare our measurements with theoretical predictions. We use data from the Australian SKA Pathfinder's EMU Main Survey, which overlaps with the Euclid Deep Field South. We generate two radio-source catalogues using different source finders to create galaxy maps. We measure the harmonic-space cross-correlation signal using a pseudo-spectrum estimator. The measured signal is compared to theoretical predictions based on a {\Lambda}CDM cosmology, using several models for the EMU source redshift distribution and bias. We report detection above 8{\sigma} of the cross-correlation signal consistent across all tested models and data sets. The measured cross-spectra from the two radio catalogues are in excellent agreement, demonstrating that the cross-correlation is robust against the choice of source-finding algorithm. The measured signal also agrees with theoretical models developed from previous cross-correlation studies and simulations. This pathfinder study establishes a statistically significant cross-correlation between EMU and Euclid. The robustness of the signal is a crucial validation of the methodology, paving the way for future large-scale analyses leveraging the full power of this synergy to constrain cosmological parameters and our understanding of galaxy evolution.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.23147-b31b1b.svg)](https://arxiv.org/abs/2511.23147) | **Sulfur monoxide as a tracer of the Galactic $^{32}$S/$^{34}$S gradient**  |
|| Y. Zou, et al. -- incl., <mark>Y. Wang</mark> |
|*Appeared on*| *2025-12-01*|
|*Comments*| *25 pages, 12 figures, 3 tables + Appendices. Accepted for publication in The Astrophysical Journal Supplement Series*|
|**Abstract**|            To date, the Galactic interstellar radial $^{32}$S/$^{34}$S gradient has only been studied with the CS isotopologs, which may be affected by uncertainties due to the use of a single tracer. As another abundant S-bearing molecules, SO and its isotopomer $^{34}$SO could be considered as tracers of the $^{32}$S/$^{34}$S ratio. We present the first systematic observations of SO and $^{34}$SO toward a large sample of molecular clouds with accurate distances, performed with the IRAM 30 m and the 10 m Submillimeter Telescope (SMT). With the IRAM 30 m, SO $2_2-1_1$ was detected in 59 of 82 sources ($\sim$82%), and $^{34}$SO $2_2-1_1$ in 8 sources ($\sim$10%). With the SMT 10 m, SO $5_5-4_4$ was detected in 136 of 184 sources ($\sim$74%), and $^{34}$SO $5_5-4_4$ in 55 of 77 strong SO sources ($\sim$72%). SO/$^{34}$SO ratios were derived for 8 ($2_2-1_1$) and 55 ($5_5-4_4$) sources. No correlation was found between the SO/$^{34}$SO ratio and heliocentric distance or $T_k$, suggesting negligible distance and fractionation effects. Both LTE and non-LTE analyses consistently suggest that the optical depth effect is also insignificant. $^{32}$S/$^{34}$S ratios from the $2_2-1_1$ transitions follow the increasing radial trend proposed by previous CS species measurements, while those from the $5_5-4_4$ lines are systematically lower. The lower transitions of SO and $^{34}$SO may be suitable tracers of $^{32}$S/$^{34}$S, though the detections are rare. Comparisons between measurements and Galactic chemical evolution model suggest that the nucleosynthesis prescriptions need to be revised in the low-metallicity regime, but more data for the outermost Galactic regions are crucial for drawing strong conclusions.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.22527-b31b1b.svg)](https://arxiv.org/abs/2511.22527) | **Spiral excitation in protoplanetary disks through gap-edge illumination: Distinctive kinematic signatures in CO isotopologues**  |
|| D. Muley, L.-A. Hühn, <mark>H. Jiang</mark>, <mark>D. M. Fuksman</mark> |
|*Appeared on*| *2025-12-01*|
|*Comments*| *18 pages, 15 figures incl. Appendix. Accepted to Astronomy and Astrophysics; further comments and questions welcome*|
|**Abstract**|            High-resolution, near-infrared observations have revealed prominent, two-armed spirals in a multitude of systems, such as MWC~758, SAO~206462, and V1247~Ori. Alongside the classical theory of disk-companion interaction, shadow-based driving has come into vogue as a potential explanation for such large-scale substructures. How might these two mechanisms be distinguished from one another in observations? To investigate this question, we ran a pair of hydrodynamical simulations with \texttt{PLUTO}. One, with full radiation hydrodynamics and gas-grain collision, was designed to develop shadow-driven spirals at the outer gap edge of a sub-thermal, Saturn-mass planet. The other, with parametrized $\beta$-cooling, was set up to capture the more standard view of spiral wave excitation by a super-thermal, multi-Jupiter-mass, exterior planetary companion. Post-processing of these simulations with the Monte Carlo radiative transfer (MCRT) code \texttt{RADMC3D} revealed that strong vertical velocities in the shadow-driven case create a prominent two-armed feature in the moment-1 CO maps, particularly when the disk is viewed face-on in optically thicker isotopologues; such a feature is not seen in the standard planet-driven case. Conversely, the presence or absence of such signatures in two-armed spiral systems would distinguish those potentially driven by exterior, multi-Jupiter-mass companions, and thus help identify promising targets for future direct-imaging campaigns.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error list index out of range</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.22472-b31b1b.svg)](https://arxiv.org/abs/2511.22472) | **The Solar Close Observations and Proximity Experiments (SCOPE) mission**  |
|| J. Lin, et al. -- incl., <mark>H. Jiang</mark>, <mark>X. Zhang</mark> |
|*Appeared on*| *2025-12-01*|
|*Comments*| *27 pages, 15 figures*|
|**Abstract**|            The Solar Close Observations and Proximity Experiments (SCOPE) mission will send a spacecraft into the solar atmosphere at a low altitude of just 5 R_sun from the solar center. It aims to elucidate the mechanisms behind solar eruptions and coronal heating, and to directly measure the coronal magnetic field. The mission will perform in situ measurements of the current sheet between coronal mass ejections and their associated solar flares, and energetic particles produced by either reconnection or fast-mode shocks driven by coronal mass ejections. This will help to resolve the nature of reconnections in current sheets, and energetic particle acceleration regions. To investigate coronal heating, the mission will observe nano-flares on scales smaller than 70 km in the solar corona and regions smaller than 40 km in the photosphere, where magnetohydrodynamic waves originate. To study solar wind acceleration mechanisms, the mission will also track the process of ion charge-state freezing in the solar wind. A key achievement will be the observation of the coronal magnetic field at unprecedented proximity to the solar photosphere. The polar regions will also be observed at close range, and the inner edge of the solar system dust disk may be identified for the first time. This work presents the detailed background, science, and mission concept of SCOPE and discusses how we aim to address the questions mentioned above.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2511.21820/./figures/HR_T_lambda_v2.png', 'tmp_2511.21820/./figures/LHa_correlations.png', 'tmp_2511.21820/./figures/sample_select_v3.png']
copying  tmp_2511.21820/./figures/HR_T_lambda_v2.png to _build/html/
copying  tmp_2511.21820/./figures/LHa_correlations.png to _build/html/
copying  tmp_2511.21820/./figures/sample_select_v3.png to _build/html/
exported in  _build/html/2511.21820.md
    + _build/html/tmp_2511.21820/./figures/HR_T_lambda_v2.png
    + _build/html/tmp_2511.21820/./figures/LHa_correlations.png
    + _build/html/tmp_2511.21820/./figures/sample_select_v3.png
found figures ['tmp_2511.22399/./plots/figure_5.png', 'tmp_2511.22399/./plots/figure_7.png', 'tmp_2511.22399/./plots/figure_10.png']
copying  tmp_2511.22399/./plots/figure_5.png to _build/html/
copying  tmp_2511.22399/./plots/figure_7.png to _build/html/
copying  tmp_2511.22399/./plots/figure_10.png to _build/html/
exported in  _build/html/2511.22399.md
    + _build/html/tmp_2511.22399/./plots/fig

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\ergs}{\rm erg s^{-1}}$
$\newcommand{\Oi}{O {\sc i}\xspace}$
$\newcommand{\Oii}{[O {\sc ii}]\xspace}$
$\newcommand{\Oiii}{[O {\sc iii}]\xspace}$
$\newcommand{\Hei}{He {\sc i}\xspace}$
$\newcommand{\Hi}{H {\sc i}\xspace}$
$\newcommand{\Hii}{H {\sc ii}\xspace}$
$\newcommand{\Nii}{[N {\sc ii}]\xspace}$
$\newcommand{\Neiii}{[Ne {\sc iii}]\xspace}$
$\newcommand{\Caii}{Ca {\sc ii}\xspace}$
$\newcommand{\Hb}{{\rm H}\beta\xspace}$
$\newcommand$
$\newcommand{\Msun}{\rm M_\odot}$
$\newcommand{\kpc}{ \rm kpc}$
$\newcommand{\kms}{ \rm km s^{-1}}$
$\newcommand{\re}{r_{\rm e,maj}}$
$\newcommand{\rec}{r_{\rm e}}$
$\newcommand{\fdm}{f_{\rm DM}}$
$\newcommand{\fbar}{f_{\rm baryon}}$
$\newcommand{\Mgas}{M_{\rm gas}}$
$\newcommand{\Mdm}{M_{\rm DM}}$
$\newcommand{\Mbar}{M_{\rm baryon}}$
$\newcommand{\Mdyn}{M_{\rmdyn}}$
$\newcommand{\Mbh}{M_{\rm BH}}$
$\newcommand{\reff}{r_{\rm e}}$
$\newcommand{\Ha}{H\alpha\xspace}$
$\newcommand{\Pad}{Pa-\delta\xspace}$
$\newcommand{\Pag}{Pa-\gamma\xspace}$
$\newcommand{\Pab}{Pa-\beta\xspace}$
$\newcommand{\Paa}{Pa-\alpha\xspace}$
$\newcommand{\AG}[1]{{\bf \color{blue}{#1} }}$
$\newcommand{\ruby}{\textit{The Cliff}\xspace}$
$\newcommand{\zspec}{z_{\rm spec}}$
$\newcommand{\zphot}{z_{\rm phot}}$
$\newcommand{\lpeak}{\lambda_{\rm peak}}$
$\newcommand{\Teff}{T_{\rm eff}}$
$\newcommand{\betaBB}{\beta_{\rm MBB}}$
$\newcommand{\buv}{\beta_{\rm UV}}$
$\newcommand{\bopt}{\beta_{\rm opt}}$
$\newcommand{\Muv}{M_{\rm UV}}$
$\newcommand{\Lopt}{L_{5100}}$
$\newcommand{\pix}{\rm pix}$
$\newcommand{\prospector}{\texttt{Prospector}\xspace}$
$\newcommand{\pysersic}{\texttt{pysersic}\xspace}$
$\newcommand{\Av}{A_{\rm V}}$
$\newcommand{\cloudy}{\texttt{Cloudy}\xspace}$
$\newcommand{\mombh}{MoM-BH*-1\xspace}$
$\newcommand$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

# Little Red Dots host Black Hole Stars: A unified family of gas-reddened AGN revealed by JWST/NIRSpec spectroscopy

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2511.21820-b31b1b.svg)](https://arxiv.org/abs/2511.21820)<mark>Appeared on: 2025-12-01</mark> -  _22 pages, 18 figures, submitted to MNRAS. LRD sample and measurements are publicly available at this https URL_

</div>
<div id="authors">

<mark>A. d. Graaff</mark>, et al. -- incl., <mark>H.-W. Rix</mark>, <mark>F. Walter</mark>

</div>
<div id="abstract">

**Abstract:** We use the DAWN JWST Archive to construct and characterise a sample of 116 little red dots (LRDs) across $\mbox{2.3<z<9.3}$ , selecting all sources with v-shaped UV-optical continua from NIRSpec/PRISM spectra and compact morphologies in NIRCam/F444W imaging. We show that LRD continuum spectra are ubiquitously well described by modified blackbodies across $\sim0.4-1.0 \micron$ , with typical $T\sim5000 $ K or $\lpeak\sim0.65 \micron$ across 2 dex in luminosity, and a tail toward $T\sim2000 $ K. LRDs therefore trace a locus in the Hertzsprung-Russell diagram that is directly analogous to stars on the Hayashi track, strongly supporting the picture that LRDs are AGN embedded in thermalised dense gas envelopes in approximate hydrostatic equilibrium. Hotter LRDs with $\lpeak<0.65 \micron$ typically have strong Balmer breaks, redder UV slopes and high optical luminosities ( $L_{5100}$ ); other LRDs show weak or no Balmer breaks at all, and wide variety in $\buv$ and $L_{5100}$ . Crucially, we demonstrate that the UV-optical continuum shapes and luminosities are strongly linked to the $\Ha$ , $\Hb$ , $\Oiii$ and $\Oi$ line properties. There is a tight linear relation between the $\Ha$ and optical continuum luminosities, as well as $\Ha$ and $\textrm{\Oi}_{\lambda8446}$ , indicating that Balmer, $\Oi$ and optical emission must primarily be powered by the same source. The Balmer decrement increases strongly toward higher $L_{\rm H\alpha}$ , $L_{5100}$ and Balmer break strength, providing key evidence for luminosity-dependent effects of collisional (de-)excitation and resonant scattering in the gaseous envelopes. In contrast, we show that $\Oiii$ emission likely originates from star-forming host galaxies, and that its strong correlation with Balmer break strength arises naturally from variation in the AGN-to-host ratio. Our work presents an empirical description of the nature and structure of LRDs, defining a new benchmark for ongoing LRD model developments.

</div>

<div id="div_fig1">

<img src="tmp_2511.21820/./figures/HR_T_lambda_v2.png" alt="Fig12" width="100%"/>

**Figure 12. -** HR diagram of LRDs at $z<4.5$ for the modified blackbody temperature (left) and peak wavelength (middle). LRDs span a wide range in luminosity, with a lower luminosity limit that may be part physical and part driven by the imposed magnitude limit and depth of the PRISM spectra. The rest-optical continua of LRDs typically peak at a wavelength of $\sim 0.65 \micron$, corresponding to a temperature of $\sim 5000 $K for a perfect blackbody ($\beta=0$), although some systems are as cold as $\sim2000 $K. The right panel shows examples of extremes in the HR diagram: a remarkably cold source ($T\sim2300 $K, $\betaBB\sim2$, $\lpeak\sim0.83 \mu$m; UNCOVER-20698), and a source that peaks in the near-IR ($\lpeak\sim1.05 \mu$m, $\betaBB\sim-2$; CAPERS-COSMOS-30440).  (*fig:HR*)

</div>
<div id="div_fig2">

<img src="tmp_2511.21820/./figures/LHa_correlations.png" alt="Fig14" width="100%"/>

**Figure 14. -** $\Ha$ line properties as a function of the optical continuum shape parameters ($\lpeak$, Balmer break) and UV continuum properties ($\buv$, $\Muv$). Large (small) circles show the $z<4.5$(full) LRD sample, and blue squares the RUBIES broad line sources of [Hviding, et. al (2025)](https://ui.adsabs.harvard.edu/abs/2025arXiv250605459H) that are not LRDs. Spearman correlation coefficients and p-values are shown in each panel for the LRD sample. The $\Ha$ emission is independent of the rest-optical continuum shape, but is correlated with the rest UV, as LRDs with the reddest UV slopes are most luminous in $\Ha$. The UV luminosities of LRDs correlate with both the $\Ha$ luminosities and EWs, and suggests that a substantial fraction of the UV can be attributed to emission from AGN.   (*fig:LHa_corr*)

</div>
<div id="div_fig3">

<img src="tmp_2511.21820/./figures/sample_select_v3.png" alt="Fig1" width="100%"/>

**Figure 1. -** Top: Redshift distribution of all PRISM spectra selected from the public DJA that satisfy the spectroscopic v-shape criteria of [Hviding, et. al (2025)](https://ui.adsabs.harvard.edu/abs/2025arXiv250605459H)(black). The sample of v-shaped objects contains both LRDs and a large number of dust-reddened star-forming galaxies at cosmic noon. We define LRDs (116 unique sources; red) as the subset of sources satisfying strict compactness criteria in NIRCam/F444W imaging. Bottom: NIRCam/F444W magnitudes measured in circular apertures of radius $0.1$\arcsec$$(half the MSA slit width) of all LRDs as a function of redshift. Of these sources, $47\%$(55/116) also have medium or high resolution spectra from NIRSpec. Dashed lines indicate the subsample used for the modified blackbody fitting of Section $\re$f{sec:MBB}. (*fig:redshifts*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2511.21820"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\rcc}[0]{\textcolor{red}}$
$\newcommand{\orcid}[1]$</div>



<div id="title">

# $\Euclid$\/ preparation: LXXXI. The impact of nonparametric star formation histories on spatially resolved galaxy property estimation using synthetic $\Euclid$ images

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2511.22399-b31b1b.svg)](https://arxiv.org/abs/2511.22399)<mark>Appeared on: 2025-12-01</mark> -  _23 pages, 12 figures, accepted_

</div>
<div id="authors">

E. Collaboration, et al. -- incl., <mark>K. Jahnke</mark>

</div>
<div id="abstract">

**Abstract:**            We analyzed the spatially resolved and global star formation histories (SFHs) for a sample of 25 TNG50-SKIRT Atlas galaxies to assess the feasibility of reconstructing accurate SFHs from Euclid-like data. This study provides a proof of concept for extracting the spatially resolved SFHs of local galaxies with Euclid, highlighting the strengths and limitations of SFH modeling in the context of next-generation galaxy surveys. We used the spectral energy distribution (SED) fitting code Prospector to model both spatially resolved and global SFHs using parametric and nonparametric configurations. The input consisted of mock ultraviolet--near-infrared photometry derived from the TNG50 cosmological simulation and processed with the radiative transfer code SKIRT. We show that nonparametric SFHs provide a more effective approach to mitigating the outshining effect by recent star formation, offering improved accuracy in the determination of galaxy stellar properties. Also, we find that the nonparametric SFH model at resolved scales closely recovers the stellar mass formation times (within 0.1~dex) and the ground truth values from TNG50, with an absolute average bias of $0.03$~dex in stellar mass and $0.01$~dex in both specific star formation rate and mass-weighted age. In contrast, larger offsets are estimated for all stellar properties and formation times when using a simple $\tau$-model SFH, at both resolved and global scales, highlighting its limitations. These results emphasize the critical role of nonparametric SFHs in both global and spatially resolved analyses, as they better capture the complex evolutionary pathways of galaxies and avoid the biases inherent in simple parametric models.         

</div>

<div id="div_fig1">

<img src="tmp_2511.22399/./plots/figure_5.png" alt="Fig6" width="100%"/>

**Figure 6. -** Comparisons of the spatially resolved stellar population properties for 6958 pixel bins in the 25 sample galaxies derived from spatially resolved SED fitting with a nonparametric SFH model. The depicted physical properties include the stellar mass surface density ($\Sigma_\star$), the SFR surface density ($\Sigma_\mathrm{SFR}$), the stellar metallicity ($Z_\star$), and the mass-weighted stellar age ($t_\mathrm{\star, mw}$). The colors indicate the average trend with the dust optical depth in the $V$ band ($\hat{\tau}_\mathrm{dust,  2}$), calculated with the {\tt LOESS} method. Each hexbin includes a minimum of five points. Contours enclose 20\%, 50\%, and 80\% of the total data. The dashed orange line represents the one-to-one relation. The Spearman’s correlation coefficient ($\rho$), measured bias (average offset $\mu$), and scatter (standard deviation $\sigma$) relative to the ground truth from TNG50 are indicated in the legend of each panel. (*fig:fig_5*)

</div>
<div id="div_fig2">

<img src="tmp_2511.22399/./plots/figure_7.png" alt="Fig8" width="100%"/>

**Figure 8. -** Examples of SFR surface density ($\Sigma_\mathrm{SFR}$) maps and SFH models for ten galaxies in our TNG50-{\tt SKIRT} Atlas sample. Left panel: $\Sigma_\mathrm{SFR}$ map for each galaxy, as estimated with {\tt Prospector}. Right panel: Comparison of various global SFH models to the ground truth SFH from TNG50. Specifically, we show the global nonparametric SFH inferred from our fiducial spatially resolved run (SFH$_\mathrm{res, np}$; orange), the global parametric SFH inferred from the spatially resolved map (SFH$_\mathrm{res, \tau}$; pink), the nonparametric SFH derived from fitting the integrated photometry (SFH$_\mathrm{glob, np}$; yellow), the global parametric SFH based on a simple $\tau$-model (SFH$_\mathrm{glob, \tau}$; blue), and the true global SFH from the TNG50 resolved map (SFH$_\mathrm{res, TNG50}$; green) (*fig:fig_7*)

</div>
<div id="div_fig3">

<img src="tmp_2511.22399/./plots/figure_10.png" alt="Fig11" width="100%"/>

**Figure 11. -** Differences in the global stellar properties for the four different SFH models with respect to the ground truth global values from the TNG50 simulation. From left to right: $M_\star$, sSFR, and $t_\mathrm{\star, mw}$. The properties were derived through SED fitting under four different configurations: spatially resolved fitting with a nonparametric SFH$_\mathrm{res, np}$(orange), spatially resolved SED fitting with a simple $\tau$-model SFH$_\mathrm{res, \tau}$(pink), global fitting with a nonparametric SFH$_\mathrm{glob, np}$(yellow), and a global fitting with a simple $\tau$-model SFH$_\mathrm{glob, \tau}$(blue). Typical uncertainties for each configuration are shown in the bottom-left corner of each panel. Star-forming galaxies are indicated with stars, while quiescent galaxies with points. The bottom panels show the residual distributions of each stellar property. The measured bias (average offset) and scatter (standard deviation) are indicated in the legends of the panels. The properties inferred from the spatially resolved and integrated nonparametric SFHs are in best agreement with the ground truth values. (*fig:fig_10*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2511.22399"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$</div>



<div id="title">

# Classical Cepheids in the Galactic thin disk.   : I. Abundance gradients through NLTE spectral analysis

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2511.22491-b31b1b.svg)](https://arxiv.org/abs/2511.22491)<mark>Appeared on: 2025-12-01</mark> -  _Manuscript submitted to A&A. Abstract shortened. Comments are welcome_

</div>
<div id="authors">

A. Nunnari, et al. -- incl., <mark>M. Bergemann</mark>

</div>
<div id="abstract">

**Abstract:** Classical Cepheids (CCs) have long been considered excellent tracers of the chemical evolution of the Milky Way's young disk. We present a homogeneous, Non-Local Thermodynamical Equilibrium (NLTE) spectroscopic analysis of 401 Galactic CCs, based on 1,351 high-resolution optical spectra, spanning Galactocentric distances from 4.6 to 29.3 kpc. Using PySME with MARCS atmospheres and state-of-the-art grids of NLTE departure coefficients, we derive atmospheric parameters and abundances for key species tracing multiple nucleosynthetic channels (O, Na, Mg, Al, Si, S, Ca, Ti, Mn, Fe, Cu). Our sample--the largest CC NLTE dataset to date-- achieves high internal precision and enables robust modeling of present-day thin-disk abundance patterns and radial gradients.  We estimate abundance gradients using three analytic prescriptions (linear, logarithmic, bilinear with a break) within a Bayesian, outlier-robust framework, and we also apply Gaussian Process Regression to capture non-parametric variations. We find that NLTE atmospheric parameters differ systematically from LTE determinations. Moreover, iron and most elemental abundance profiles are better described by non-linear behavior rather than by single-slope linear models: logarithmic fits generally outperform simple linear models, while bilinear fits yield inconsistent break radii across elements.  Gaussian Process models reveal a consistent outer-disk flattening of [ X/H ] for nearly all studied elements. The [ X/Fe ] ratios are largely flat with Galactocentric radius, indicating coherent chemical scaling with iron across the thin disk, with modest positive offsets for Na and Al and mild declines for Mn and Cu.  Finally, Cepheid kinematics confirm thin-disk orbits for the great majority of the sample. Comparison with recent literature shows overall agreement but highlights NLTE-driven differences, especially in outer-disk abundances. These results provide tighter empirical constraints for chemo-dynamical models of the Milky Way and set the stage for future NLTE mapping with upcoming large spectroscopic surveys.

</div>

<div id="div_fig1">

<img src="tmp_2511.22491/./fig07_Fe_m_b_LIN_LOG_BIL_residuals.png" alt="Fig5" width="100%"/>

**Figure 5. -** \small{Panel a) -- Iron radial gradient for classical Cepheids as a function of the
      Galactocentric distance. The linear, the logarithmic and the bilinear fits are shown respectively
      in blue, red and green. The vertical green dashed line shows the "knee" of the bilinear fit.
      The coefficients of the fits are labelled.
      Panel b) -- residuals of the linear fit. The values of the mean square error, the reduced chi-squared
      and the AIC score are labelled.
      Panel c) -- Same as panel b), but for the logarithmic fit.
      Panel d) -- Same as panel b), but for the bilinear fit.} (*iron*)

</div>
<div id="div_fig2">

<img src="tmp_2511.22491/./fig_lte_teff_new.png" alt="Fig2.1" width="16%"/><img src="tmp_2511.22491/./fig_lte_logg_new.png" alt="Fig2.2" width="16%"/><img src="tmp_2511.22491/./fig_lte_vmic_new.png" alt="Fig2.3" width="16%"/><img src="tmp_2511.22491/./fig_lte_vmac_new.png" alt="Fig2.4" width="16%"/><img src="tmp_2511.22491/./fig_lte_feh_new.png" alt="Fig2.5" width="16%"/><img src="tmp_2511.22491/./fig_lte_ti_new.png" alt="Fig2.6" width="16%"/>

**Figure 2. -** Comparison of atmospheric parameters derived under NLTE and LTE assumptions. Each panel displays the difference (NLTE – LTE) as a function of the corresponding parameter on the left, and the distribution of the difference on the right. A blue-dotted Gaussian fit is overplotted on each histogram, the mean and standard deviation are labelled in the top-right corner. The blue-dashed horizontal line shows the mean difference, while the gray solid line the null difference. (*lte_plots*)

</div>
<div id="div_fig3">

<img src="tmp_2511.22491/./fig14_FeH_vs_RGC_GPR_bis.png" alt="Fig8" width="100%"/>

**Figure 8. -** In gray iron abundances of CCs as a function of $\mathrm{R_{GC}$. The orange symbols display open clusters younger than 400 Myr from \citealt{otto25}. GPR applied to the CCs distribution is highlighted in blue, while the shaded region highlights the uncertainties on the model.} (*gpr*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2511.22491"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand{\pcc}{ cm^{-3}}$
$\newcommand{\add}{ }$
$\newcommand{\addmore}$
$\newcommand$
$\newcommand{\fnum@figure}{\textbf{Figure \thefigure}}$
$\newcommand{\fnum@table}{\textbf{Table \thetable}}$
$\newcommand{\thefigure}{S\arabic{figure}}$
$\newcommand{\thetable}{S\arabic{table}}$
$\newcommand{\theequation}{S\arabic{equation}}$
$\newcommand{\thepage}{S\arabic{page}}$
$\newcommand\scititle{$
$	Large-amplitude Variability Driven by Giant Dust Storms on a Planetary-mass Companion$
$}$</div>



<div id="title">

# $\bfseries$ $\boldmath$ $\scititle$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2511.23163-b31b1b.svg)](https://arxiv.org/abs/2511.23163)<mark>Appeared on: 2025-12-01</mark> -  _Science Advances 2025 Nov. 26;11(48). The simulation data in this work are publicly available at this http URL_

</div>
<div id="authors">

X. Tan, et al. -- incl., <mark>X. Zhang</mark>, <mark>G. Chauvin</mark>

</div>
<div id="abstract">

**Abstract:** $\bfseries$ $\boldmath$ Large-amplitude variations are commonly observed in the atmospheres of directly imaged exoplanets and brown dwarfs. VHS 1256B, the most variable known planet-mass object, exhibits a near-infrared flux change of nearly 40 \% , with red color and silicate features revealed in recent JWST spectra, challenging current theories. Using a general circulation model, we demonstrate that VHS 1256B’s atmosphere is dominated by planetary-scale dust storms persisting for tens of days, with large patchy clouds propagating with equatorial waves. This weather pattern, distinct from the banded structures seen on solar system giants, simultaneously explains the observed spectra and critical features in the rotational light curves, including the large amplitude, irregular evolution, and wavelength dependence, as well as the variability trends observed in near-infrared color-magnitude diagrams of ${\add dusty}$ substellar atmospheres.

</div>

<div id="div_fig1">

<img src="tmp_2511.23163/./figures/summary_version2-4nec2.png" alt="Fig3" width="100%"/>

**Figure 3. -** **Model spectroscopic light curves, spectrum, and comparisons to the JWST data.**{$\footnote$size _Panels A, B, and C:_ synthetic light curves from the GCM in three epochs of outputs corresponding to three characteristic evolution stages of the equatorial dust storm. Blue lines are light curves running through the HST/WFC3 G141 filter and the red lines are those running through the Spitzer 4.5 $\mu$m filter. _Panel D:_  model spectrum of VHS 1256B based on the GCM outputs at the time when the light curve reaches the minimum at epoch 1, with a spectral resolution of $R\sim 3000$ (black line), and the observed spectrum of VHS 1256B obtained by JWST (colored lines) \cite{miles2023}. In the radiative transfer post-processing, we have slightly altered the gas compositions of $H_2$O, CO and $CH_4$ taking into account the possible chemical disequilibrium caused by dynamical mixing \cite{miles2023}. The JWST spectrum is scaled for a direct comparison with the model spectrum, and different colors are those measured by different instruments, and the grey lines represent error bars. Important molecular gas and silicate cloud absorbers are highlighted. _Panels E, F, G, and H:_ zoom-in of the spectra in a few interesting wavelength regions: panel E contains $H_2$O and alkali metal absorption; panel F shows primarily the $CH_4$ absorption; panel G highlights the CO absorption; and panel H exhibits the silicate cloud absorption.} (*fig.summary*)

</div>
<div id="div_fig2">

<img src="tmp_2511.23163/./figures/bolo_map_lightcurve_v4.png" alt="Fig1" width="100%"/>

**Figure 1. -** **Bolometric thermal flux maps, their time evolution near the equator, and the resulting long-term light-curve evolution from the GCM.**{$\add$ _Panel A, B, C:_ Top-of-atmosphere (TOA) bolometric flux maps from the GCM at different epochs, showing global-scale dust storms (the dark regions) at the equator. }_Panel D:_ the Hovmoller diagram (longitude-time sequence) of the TOA bolometric flux near the equator. The flux is averaged over latitudes within $\pm 10$ degrees to cover a fraction of the meridionally extended waves. The dark features evolving towards the upper right mean the eastward propagation of the dust storms.  _Panel E:_ the long-term white light curve of the GCM (black line). The red line is the light curve with periodic signals less than 4 rotation periods filtered out. The much smaller amplitude of the filtered light curve suggests that the variability (black line) is mostly rotational rather than from an intrinsic variation of the mean atmospheric state. Epochs 1, 2, and 3 enclosed in the panel correspond to {$\add$ a few different characteristic states of the equatorial waves}. {$\add$ _Panel F:_ a zoom-in of the light curve between elapsed time of 4400 to 6500 hours.} (*fig.whitelight*)

</div>
<div id="div_fig3">

<img src="tmp_2511.23163/./figures/summary_allother.png" alt="Fig17" width="100%"/>

**Figure 17. -** $\add$**Summary plots from GCM tests with one silicate mode of $r_0=7$, $1$, and $5 \mu$m, as well as for a test with the same silicate cloud setup as in the main text but excluding Fe clouds in the GCM.**_Panels A, B, C and D:_ instantaneous global thermal flux maps for different tests. _Panels E, F, G, and H:_ Data-model comparison of the spectra. _Panels I, J, K and L:_ normalized spectroscopic light curves at a few selected wavelengths.
     (*fig.summary_all*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2511.23163"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

135  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

13  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

4  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
